In [3]:
import time
import torch
from vmas import make_env
from vmas.simulator.core import Agent
from vmas.simulator.scenario import BaseScenario
from typing import Union
from moviepy.editor import ImageSequenceClip
from IPython.display import HTML, display as ipython_display

class VMASEnvRunner:
    def __init__(
        self,
        render: bool,
        num_envs: int,
        n_steps: int,
        device: str,
        scenario: Union[str, BaseScenario],
        continuous_actions: bool,
        random_action: bool,
        **kwargs
    ):
        self.render = render
        self.num_envs = num_envs
        self.n_steps = n_steps
        self.device = device
        self.scenario = scenario
        self.continuous_actions = continuous_actions
        self.random_action = random_action
        self.kwargs = kwargs
        self.frame_list = []  # Initialize frame list

    def _get_deterministic_action(self, agent: Agent, env):
        if self.continuous_actions:
            action = -agent.action.u_range_tensor.expand(env.batch_dim, agent.action_size)
        else:
            action = torch.tensor([1], device=env.device, dtype=torch.long).unsqueeze(-1).expand(env.batch_dim, 1)
        return action.clone()

    def generate_gif(self, scenario_name):
        fps = 30
        clip = ImageSequenceClip(self.frame_list, fps=fps)
        clip.write_gif(f'{scenario_name}.gif', fps=fps)

        # Return the HTML tag to display the GIF
        return HTML(f'<img src="{scenario_name}.gif">')

    def run_vmas_env(self):
        scenario_name = self.scenario if isinstance(self.scenario, str) else self.scenario.__class__.__name__

        env = make_env(
            scenario=self.scenario,
            num_envs=self.num_envs,
            device=self.device,
            continuous_actions=self.continuous_actions,
            seed=0,
            **self.kwargs
        )

        init_time = time.time()
        step = 0

        for s in range(self.n_steps):
            step += 1
            print(f"Step {step}")

            actions = []
            for i, agent in enumerate(env.agents):
                if not self.random_action:
                    action = self._get_deterministic_action(agent, env)
                else:
                    action = env.get_random_action(agent)

                actions.append(action)

            obs, rews, dones, info = env.step(actions)

            if self.render:
                frame = env.render(
                    mode="rgb_array",
                    agent_index_focus=None,
                )
                self.frame_list.append(frame)  # Append frame to the list

        total_time = time.time() - init_time
        print(
            f"It took: {total_time}s for {self.n_steps} steps of {self.num_envs} parallel environments on device {self.device} "
            f"for {scenario_name} scenario."
        )

if __name__ == "__main__":

    scenario_name = "waterfall"

    env_runner = VMASEnvRunner(
        render=True,
        num_envs=32,
        n_steps=10,
        device="cuda",
        scenario=scenario_name,
        continuous_actions=False,
        random_action=False,
        # Environment specific variables
        n_agents=2,
    )
    # Run the VMAS environment
    env_runner.run_vmas_env()

    # Generate and display the GIF
    ipython_display(env_runner.generate_gif(scenario_name))


Step 1
Step 2
Step 3
Step 4
Step 5
Step 6
Step 7
Step 8
Step 9
Step 10
It took: 1.7386164665222168s for 10 steps of 32 parallel environments on device cuda for waterfall scenario.
MoviePy - Building file waterfall.gif with imageio.
